### Introduction

Continuous Stirred-Tank Reactors (CSTRs) are essential components in the chemical industry, utilized for reactions requiring continuous input and output of materials. Optimizing the operation of a CSTR involves balancing reaction rates, energy consumption, and output quality under specific constraints. This case study explores the optimization of a CSTR undergoing a second-order exothermic reaction, focusing on maximizing conversion while managing energy usage.

### Scenario Description

#### Chemical Reaction:
Consider a second-order exothermic reaction in a CSTR:

A + B → C

Where:

- **A** and **B** are the reactants.
- **C** is the desired product.

#### Reaction Kinetics:
The rate of reaction is given by:

r = k * C_A * C_B

Where:

- **r** is the rate of reaction (mol/L·s).
- **C_A** and **C_B** are the concentrations of reactants A and B (mol/L).
- **k** is the rate constant (L/mol·s).

The rate constant \( k \) follows the Arrhenius equation:

k = k₀ * exp(-Eₐ / (R * T))

Where:

- **k₀** is the pre-exponential factor (L/mol·s).
- **Eₐ** is the activation energy (J/mol).
- **R** is the universal gas constant (8.314 J/mol·K).
- **T** is the temperature (K).

### Feed Conditions:
- **Inlet concentration of A (C_A0)**: 1 mol/L
- **Inlet concentration of B (C_B0)**: 1 mol/L
- **Feed flow rate (F)**: 2 L/s
- **Inlet temperature (T₀)**: 300 K

### Operating Conditions:
- **Reactor volume (V)**: 5 L
- The reactor includes a cooling jacket to manage the exothermic reaction.
- **Heat transfer coefficient (U)**: 600 W/m²·K
- **Jacket area (A_j)**: 1.5 m²
- **Coolant temperature (T_c)**: 280 K

### Constraints:
- The reactor temperature should not exceed 350 K to prevent unwanted side reactions.
- The concentration of reactant B in the effluent must be at least 0.05 mol/L to maintain process efficiency.


### Equations and Procedures

#### Governing Equations

**Mass Balance for Component A:**

\[ F \times C_{A0} - F \times C_A - V \times r = 0 \]

**For Component B:**

\[ F \times C_{B0} - F \times C_B - V \times r = 0 \]

**Substituting \( r = k \times C_A \times C_B \):**

For A:

\[ F \times C_{A0} - F \times C_A - V \times k \times C_A \times C_B = 0 \]

For B:

\[ F \times C_{B0} - F \times C_B - V \times k \times C_A \times C_B = 0 \]

#### Energy Balance:

\[ F \times C_p \times (T_0 - T) + V \times (-\Delta H_r) \times k \times C_A \times C_B - U \times A_j \times (T - T_c) = 0 \]

Where:

- \( C_p \) is the specific heat capacity (J/mol·K).
- \( \Delta H_r \) is the heat of reaction (J/mol).

**Reaction Rate (Arrhenius Equation):**

\[ k = k_0 \times \exp\left(-\frac{E_a}{R \times T}\right) \]

#### Optimization Problem

**Objective Function:**

The primary objective is to maximize the conversion of reactant A:

\[ X_A = \frac{C_{A0} - C_A}{C_{A0}} \]

Alternatively, you may want to minimize the energy consumption represented by the heat removed by the cooling jacket:

\[ Q = U \times A_j \times (T - T_c) \]

**Constraints:**

- **Concentration Constraint:** \( C_B \geq 0.05 \) mol/L
- **Temperature Constraint:** \( T \leq 350 \) K
- **Reaction Rate Constant (from Arrhenius equation):**

\[ k = k_0 \times \exp\left(-\frac{E_a}{R \times T}\right) \]

#### Procedure for Setting up the Optimization Problem

1. **Define Variables:**

   - \( T \) (Reactor Temperature)
   - \( C_A \) (Concentration of A)
   - \( C_B \) (Concentration of B)
   - \( X_A \) (Conversion of A)
   - \( Q \) (Heat removal rate)

2. **Formulate Objective Function:**

   Depending on the goal, formulate either:
   
   - **Maximization:** \( X_A \)
   - **Minimization:** \( Q \)

3. **Incorporate Constraints:**

   Add constraints for temperature, concentration, and the reaction rate.

4. **Non-linearities:**

   Handle non-linear terms in the Arrhenius equation and energy balance through linearization or by using non-linear solvers.


In [5]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Define constants
C_A0 = 1.0       # Inlet concentration of A (mol/L)
U = 500          # Heat transfer coefficient (W/m²·K)
A_j = 1.0        # Jacket area (m²)
T_c = 290        # Coolant temperature (K)
k_0 = 1.0        # Pre-exponential factor (s⁻¹)
E_a = 50000.0    # Activation energy (J/mol)
R = 8.314        # Universal gas constant (J/mol·K)

# Create a new model
model = gp.Model("CSTR_Optimization")

# Add variables
T = model.addVar(lb=300, ub=350, name="Reactor_Temperature")
C_A = model.addVar(lb=0, ub=1, name="Concentration_A")
C_B = model.addVar(lb=0.05, ub=1, name="Concentration_B")
X_A = model.addVar(lb=0, ub=1, name="Conversion_A")
Q = model.addVar(lb=0, name="Heat_Removal")
k = model.addVar(name="Reaction_Rate_Constant")

# Set the objective to maximize conversion
model.setObjective(X_A, GRB.MAXIMIZE)

# Add constraints
model.addConstr(C_A == C_A0 - X_A * C_A0, name="Concentration_A_Balance")
model.addConstr(C_B >= 0.05, name="Concentration_B_Minimum")
model.addConstr(T <= 350, name="Temperature_Limit")
model.addConstr(Q == U * A_j * (T - T_c), name="Heat_Removal_Equation")

# Piecewise Linear Approximation for the Arrhenius equation
# Define breakpoints for T and corresponding k values using the Arrhenius equation
T_values = np.linspace(300, 350, 5)  # 5 breakpoints for simplicity
k_values = k_0 * np.exp(-E_a / (R * T_values))

# Add the piecewise linear constraint for k
model.addGenConstrPWL(T, k, T_values.tolist(), k_values.tolist(), "Arrhenius_PWL")

# Optimize the model
model.optimize()

# Check if the optimization was successful and print results
if model.status == GRB.OPTIMAL:
    optimal_temperature = T.x
    optimal_concentration_A = C_A.x
    optimal_concentration_B = C_B.x
    optimal_conversion = X_A.x
    optimal_heat_removal = Q.x
    
    print(f"Optimal Temperature: {optimal_temperature} K")
    print(f"Optimal Concentration of A: {optimal_concentration_A} mol/L")
    print(f"Optimal Concentration of B: {optimal_concentration_B} mol/L")
    print(f"Optimal Conversion: {optimal_conversion}")
    print(f"Optimal Heat Removal: {optimal_heat_removal} W")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 6 columns and 6 nonzeros
Model fingerprint: 0xc040e4cd
Model has 1 general constraint
Variable types: 6 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-02, 4e+02]
  RHS range        [5e-02, 1e+05]
  PWLCon x range   [3e+02, 3e+02]
  PWLCon y range   [4e-09, 2e-08]
Presolve added 4 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 8 columns, 20 nonzeros
Variable types: 5 continuous, 3 integer (3 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 1 

Optimal solution 